# Рекомендация тарифов

## Введение

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

      
**Цель проекта:** проанализировать данные о поведении клиентов оператора мобильной связи «Мегалайн» и построить ML-модель с долей правильных ответов не менее 0.75

**Данные:** 
    
  - Данные оператора мобильной связи «Мегалайн» о клиентах:  
      
    - сalls — количество звонков; 
    - minutes — суммарная длительность звонков в минутах; 
    - messages — количество sms-сообщений;
    - mb_used — израсходованный интернет-трафик в Мб;
    - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
     
**План выполнения проекта:**

  - Загрузить и изучить данные;
  - Разбить данные на выборки;
  - Исследовать качество предсказания разных моделей метрикой accuracy;
  - Проверить accuracy на тестовой выборке;
  - Проверить модель на вменяемость.
  - Сформулировать и оформить промежуточный и общий выводы.

## Загрузка и изучение данных

In [1]:
# обновляем библиотеки
!pip install -U scikit-learn -q

In [2]:
# загружаем библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

# задаем константы
RAND_ST = 12345

# загружаем данные 
data = pd.read_csv('/datasets/users_behavior.csv', sep=',')

In [3]:
# смотрим общее инфо
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# выводим первые и последние строки
display(data.head())
data.tail()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


### Вывод по первому шагу

  - **Данные предобработаны**, пропусков нет, тип данных указан верно, данные распределены по категориям 1, 0 (колонка is_ultra).

## Разделение данных на выборкии

In [5]:
# делим датасет на три выборки(тренировочную(70%), валидационную(15%) и тестовую(15%)
data_train, data_test1 = train_test_split(data, 
                                          test_size=0.3, 
                                          random_state=RAND_ST)

data_test, data_val=train_test_split(data_test1, 
                                     test_size=0.5, 
                                     random_state=RAND_ST)
# проверяем
print(data_train.shape)
print(data_test.shape)
print(data_val.shape)

(2249, 5)
(482, 5)
(483, 5)


In [6]:
# выделяем в каждой выборке признаки и целевой признак (создаем переменные)
data_train_features = data_train.drop(['is_ultra'], axis=1)
data_train_target = data_train['is_ultra']

data_val_features = data_val.drop(['is_ultra'], axis=1)
data_val_target = data_val['is_ultra']

data_test_features = data_test.drop(['is_ultra'], axis=1)
data_test_target = data_test['is_ultra']

# проверяем
print(data_train_features.shape)
print(data_train_target.shape)
print(data_val_features.shape)
print(data_val_target.shape)
print(data_test_features.shape)
print(data_test_target.shape) 

(2249, 4)
(2249,)
(483, 4)
(483,)
(482, 4)
(482,)


### Вывод по второму шагу

  - **Разбили** датасет **на три выборки**: тренировочную - **70%** данных, валидационную - **15%** и тестовую - **15%**.
   
   
  - **Выделили** из каждой выборки датафрейм с **признаками** и  колонку с **целевым признаком**. 

## Исследование моделей

### Подбор гиперпараметров

In [7]:
# выбираем лучшие гиперпараметры для Дерева решений 
# создаем списки 
best_model = None
best_result = 0
best_depth_tree = 0
best_min_samples_split = 0
best_min_samples_leaf = 0

# задаем цикл для выбора гиперпараметров 
for depth in range(1,6):
    for leaf in range(1,8):
        for split in range(2,10,2):
            # инициализируем модель
            model = DecisionTreeClassifier(random_state=12345,
                                       min_samples_split = split, 
                                       min_samples_leaf = leaf,
                                       max_depth = depth)
            # обучаем модель на тренировочной выборке
            model.fit(data_train_features, data_train_target)
            # получаем предсказания модели на валидационной выборке
            predictions_valid = model.predict(data_val_features)
            # считаем значение метрики accuracy на валидационной выборке 
            result_valid = accuracy_score(data_val_target, predictions_valid)
            # выбираем лучшие гиперпараметры
            if result_valid > best_result:
                best_model = model
                best_result = result_valid
                best_depth_tree = depth
                best_min_samples_split = split
                best_min_samples_leaf = leaf
# выводим лучшие значения accuracy и гиперпараметров      
print("Accuracy наилучшей модели 'Дерева решений' на валидационной выборке:", best_result)
print("Максимальная глубина:", best_depth_tree)  
print("Минимальное количество примеров для разделений:", best_min_samples_split)
print("Минимальное количество объектов в листе:", best_min_samples_leaf)              

Accuracy наилучшей модели 'Дерева решений' на валидационной выборке: 0.7888198757763976
Максимальная глубина: 3
Минимальное количество примеров для разделений: 2
Минимальное количество объектов в листе: 1


In [8]:
# выбираем лучший алгоритм и максимальное число итераций для Логистической регрессии 
# создаем списки
best_model = None
best_result = 0
best_max_iter = 0
best_solver = []
solvers = ['lbfgs','newton-cg', 'liblinear']

# задаем число максимальных итераций от 100 до 2000 с шагом 50
# и алгоритм
for max_iter in range(100, 2001, 50):
    for solver in solvers:
        # инициализируем модель
        model = LogisticRegression(random_state=12345, 
                                   solver=solver, 
                                   max_iter=max_iter)
        # обучаем модель на тренировочной выборке
        model.fit(data_train_features, data_train_target)
        # получаем предсказания модели на валидационной выборке 
        predictions_valid = model.predict(data_val_features)
        # считаем значение метрики accuracy на валидационной выборке 
        result_valid = accuracy_score(data_val_target, predictions_valid)
        # выбираем лучшие гиперпараметры max_iter и solver
        if result_valid > best_result:
            best_model = model
            best_result = result_valid
            best_max_iter = max_iter
            best_solver = solver            
                
# выводим лучшие значения accuracy, количества итераций и лучший алгоритм 
print("Accuracy наилучшей модели 'Логистической регрессии' на валидационной выборке:", best_result)
print("Количество итераций:", max_iter)
print("Алгоритм:", best_solver)

Accuracy наилучшей модели 'Логистической регрессии' на валидационной выборке: 0.7412008281573499
Количество итераций: 2000
Алгоритм: lbfgs


In [9]:
# выбираем лучший гиперпараметр глубины, количества деревьев и критерий (entropy, gini)
# для Случайного леса 

# создаем списки
best_model = None
best_result = 0
best_est = 0
best_depth_forest = 0
best_criterion = []
criterions = ['entropy','gini']

# задаем гиперпараметры количества деревьев до 50 с шагом 10,
# глубины от 1 до 10
# и критерий: entropy или gini
for est in range(10, 51, 10):
    for depth in range (1, 11):
        for criterion in criterions:
            # инициализируем модель
            model = RandomForestClassifier(random_state=12345,
                            n_estimators=est,
                            max_depth=depth,
                            criterion=criterion)
            # обучаем модель на тренировочной выборке
            model.fit(data_train_features, data_train_target)
            # получаем предсказания модели на валидационной выборке 
            predictions_valid = model.predict(data_val_features)
            # считаем значение метрики accuracy на валидационной выборке 
            result_valid = accuracy_score(data_val_target, predictions_valid)
            # выбираем модель с гиперпараметрами лучшего accuracy
            if result_valid > best_result:
                best_model = model
                best_result = result_valid
                best_est = est
                best_depth_forest = depth
                best_criterion = criterion
# выводим лучшее значение accuracy, количества деревьев, глубины и критерия      
print("Accuracy наилучшей модели 'Случайного леса' на валидационной выборке:", best_result)
print("Количество деревьев:", best_est)
print("Максимальная глубина:", best_depth_forest)
print("Критерий:", best_criterion)

Accuracy наилучшей модели 'Случайного леса' на валидационной выборке: 0.8260869565217391
Количество деревьев: 40
Максимальная глубина: 8
Критерий: gini


### Вывод по третьему шагу: сравнение  **accuracy** разных моделей

  - Исследовали модель **"Дерево решений"** с **разными** гиперпараметрами:
     
     - наилучшее качество (**0.7888198757763976**) показала модель с **глубиной дерева 3**, минимальным количеством примеров для разделений: 2 и минимальным количеством объектов в листе: 1.
     
 
  - Исследовали модель **"Логистическая регрессия"** с гиперпараметрами  **solver** и **max_iter**:
     
     - наилучшее качество (**0.7412008281573499**) показала модель с количеством **итераций 2000** и алгоритмом **"newton-cg"**.  


  - Исследовали модель **"Случайный лес"** с гиперпараметрами **n_estimators, max_depth** и **criterion**:
  
     - наилучшее качество (**0.8260869565217391**) показала модель с количеством **деревьев 40**, **глубиной** дерева **8** и критерием **"gini"**. 


  - **Лучшее accuracy** среди всех моделей на валидационной выборке показала модель **"Случайный лес"**

## Проверка моделей на тестовой выборке

### Проверка  accuracy моделей c  отобранными гиперпараметрами на тестовой выборке

In [10]:
# инициализируем модель дерева решений с отобранными гиперпараметрами
model = DecisionTreeClassifier(random_state=12345, 
                               max_depth = best_depth_tree,
                               min_samples_split = best_min_samples_split, 
                               min_samples_leaf = best_min_samples_leaf)

# обучаем модель на тренировочной выборке
model.fit(data_train_features, data_train_target)

# предсказываем на тренировочной выборке 
train_predictions = model.predict(data_train_features)
# рассчитываем accuracy на тренировочной выборке
accuracy_train_tree = accuracy_score(data_train_target, train_predictions)

# предсказываем на валидацинной выборке 
valid_predictions = model.predict(data_val_features)
# рассчитываем accuracy на валидационной выборке
accuracy_valid_tree = accuracy_score(data_val_target, valid_predictions)

# предсказываем на тестовой выборке 
test_predictions = model.predict(data_test_features)
# рассчитываем accuracy на тестовой выборке
accuracy_test_tree = accuracy_score(data_test_target, test_predictions)

# сравниваем результаты
print("Accuracy 'Дерева решений'")
print("Тренировочная выборка:", accuracy_train_tree) 
print("Валидационная выборка:", accuracy_valid_tree) 
print("Тестовая выборка:", accuracy_test_tree)

Accuracy 'Дерева решений'
Тренировочная выборка: 0.8016896398399288
Валидационная выборка: 0.7888198757763976
Тестовая выборка: 0.7863070539419087


In [11]:
# инициализируем модель логистической регрессии с отобранными гиперпараметрами
model = LogisticRegression(random_state=12345, 
                           solver=best_solver, 
                           max_iter=best_max_iter)

# обучаем модель на тренировочной выборке
model.fit(data_train_features, data_train_target)

# предсказываем на тренировочной выборке 
train_predictions = model.predict(data_train_features)
# рассчитываем accuracy на тренировочной выборке
accuracy_train_logistic = accuracy_score(data_train_target, train_predictions)

# предсказываем на валидацинной выборке 
valid_predictions = model.predict(data_val_features)
# рассчитываем accuracy на валидационной выборке
accuracy_valid_logistic = accuracy_score(data_val_target, valid_predictions)

# предсказываем на тестовой выборке 
test_predictions = model.predict(data_test_features)
# рассчитываем accuracy на тестовой выборке
accuracy_test_logistic = accuracy_score(data_test_target, test_predictions)

# сравниваем результаты
print("Accuracy 'Логистической регрессии'")
print("Тренировочная выборка:", accuracy_train_logistic) 
print("Валидационная выборка:", accuracy_valid_logistic) 
print("Тестовая выборка:", accuracy_test_logistic)

Accuracy 'Логистической регрессии'
Тренировочная выборка: 0.7412183192530013
Валидационная выборка: 0.7412008281573499
Тестовая выборка: 0.7717842323651453


In [12]:
# инициализируем модель случайного леса с отобранными гиперпараметрами
model = RandomForestClassifier(random_state=12345,
                        n_estimators=best_est,
                        max_depth= best_depth_forest,
                        criterion = best_criterion)
# обучаем модель на тренировочной выборке
model.fit(data_train_features, data_train_target)

# предсказываем на тренировочной выборке 
train_predictions = model.predict(data_train_features)
# рассчитываем accuracy на тренировочной выборке
accuracy_train_forest = accuracy_score(data_train_target, train_predictions)

# предсказываем на валидацинной выборке 
valid_predictions = model.predict(data_val_features)
# рассчитываем accuracy на валидационной выборке
accuracy_valid_forest = accuracy_score(data_val_target, valid_predictions)

# предсказываем на тестовой выборке 
test_predictions = model.predict(data_test_features)
# рассчитываем accuracy на тестовой выборке
accuracy_test_forest = accuracy_score(data_test_target, test_predictions)

# сравниваем результаты
print("Accuracy 'Cлучайного леса'")
print("Тренировочная выборка:", accuracy_train_forest) 
print("Валидационная выборка:", accuracy_valid_forest) 
print("Тестовая выборка:", accuracy_test_forest)

Accuracy 'Cлучайного леса'
Тренировочная выборка: 0.8723877278790574
Валидационная выборка: 0.8260869565217391
Тестовая выборка: 0.8008298755186722


In [13]:
# сравниваем Accuracy разных моделей на тестовой выборке
print("Accuracy 'Случайного леса':", accuracy_test_forest)
print("Accuracy 'Дерева решений':", accuracy_test_tree)
print("Accuracy 'Логистической регрессии':", accuracy_test_logistic)

Accuracy 'Случайного леса': 0.8008298755186722
Accuracy 'Дерева решений': 0.7863070539419087
Accuracy 'Логистической регрессии': 0.7717842323651453


### Вывод по четвертому шагу: сравнение accuracy моделей на тестовой выборке

   - В результате прогона моделей на тестовой выборке и сравнения с обучением и валидацией получилось такая картина: 
  
  
   - **Наибольшая accuracy** на **тренировочной выборке** - у модели **"Случайный лес"** - **0.8723877278790574**, при этом **не все** модели набрали **больше 0.75**: **"Логистическая регрессия"** - всего **0.7412183192530013**.
    
    
   - **На валидационной** выборке результаты теста не отличаются распределением итогов - только результатами:
       **на первом месте** снова **"Случайный лес"** - **0.8260869565217391**.
    
    
   - **На тестовой выборке** лидером, предсказуемо, стала модель **"Случайный лес"** - **0.8008298755186722**. На последнем месте **"Логистическая регрессия"** -  **0.7717842323651453**, на втором - **"Дерево решений"** - **0.7863070539419087**.
    
    
   - **У всех** моделей доля правильных ответов на тестовой выборке **больше 0.75**.

## Проверка моделей на адекватность

- **Проверим модели на вменяемость**: сравним с DummyClassifier.

In [14]:
# инициализируем константную модель
model = DummyClassifier(strategy="most_frequent") 
# обучаем модель на тренировочной выборке
model.fit(data_train_features, data_train_target)
# предсказываем на тестовой выборке 
dummy_test_predictions = model.predict(data_test_features)
#рассчитываем accuracy на тестовой выборке
accuracy_dummy = accuracy_score(data_test_target, dummy_test_predictions)
#выводим значение accuracy
accuracy_dummy

0.6950207468879668

### Вывод по пятому шагу

  - Проверка показала, что **все модели вменяемы**: у **модели-"дурочки"**, чьи ответы формируются случайным образом, **доля правильных ответов** существенно меньше, чем у любой другой модели: **0.6950207468879668**.

## Общий вывод

 -  В процессе работы над проектом мы: 
   
   - ознакомились с данными;
   - успешно разбили их на выборки;
   - исследовали качество разных моделей;
   - проверили качество на тестовой выборке;
   - проверили модели на вменяемость.
   
   
 - **Лучший показатель** accuracy (**0.8008298755186722**) в итоге получился у модели **"Случайный лес"** с количеством **деревьев: 40**, максимальной **глубиной: 8**, **критерием: "gini"** и **random_state: 12345**. Модель оказалась вменяемой.